In [ ]:
import pandas as pd

# Cargar archivos
cobros = pd.read_csv("/Users/alvarobolanos/Desktop/DataThon/Reto Credifiel/ExtraccionDomiVersionFinal/ListaCobroDetalleUnido.csv")
estrategias = pd.read_csv("/Users/alvarobolanos/Desktop/DataThon/Reto Credifiel/ExtraccionDomiVersionFinal/ListaCobroEmisora.csv")
respuestas = pd.read_csv("/Users/alvarobolanos/Desktop/DataThon/Reto Credifiel/ExtraccionDomiVersionFinal/CatRespuestaBancos.csv")

# Enriquecer con estrategia usada
cobros = cobros.merge(estrategias, on="idListaCobro", how="left")

# Enriquecer con descripción de respuesta del banco
cobros = cobros.merge(respuestas, left_on="idRespuestaBanco", right_on="IdRespuestaBanco", how="left")

# Marcar si fue exitoso
cobros["esExitoso"] = cobros["idRespuestaBanco"] == "00"

# Clasificar respuesta en categorías generales
def clasificar_respuesta(row):
    if row["idRespuestaBanco"] == "00":
        return "exitosa"
    elif row["Descripcion"] in ["Cuenta Cancelada", "Cuenta Bloqueada", "Cuenta Inexistente"]:
        return "cuenta_invalida"
    else:
        return "otros"

cobros["respuestaCategoria"] = cobros.apply(clasificar_respuesta, axis=1)

# Sort por esExitoso True 
cobros = cobros.sort_values(by=["esExitoso", "idListaCobro", "idRespuestaBanco"], ascending=[False, True, True])

cobros.head()


In [ ]:
print(cobros.shape)
# Drop columnas innecesarias: idListaCobro, fechaCobroBanco, IdRespuestaBanco
cobros.drop(columns=["idListaCobro", "fechaCobroBanco", "IdRespuestaBanco"], inplace=True)
# Drop NaN values
cobros.dropna(subset=["idRespuestaBanco"], inplace=True)
print(cobros.shape)

In [ ]:
#  mostrar valores unicos en idEmisora
print(cobros["idEmisora"].unique())

In [ ]:
# Cargar tabla de costos por estrategia
costos = pd.read_csv("/Users/alvarobolanos/Desktop/DataThon/Reto Credifiel/ExtraccionDomiVersionFinal/CatEmisora.csv")

# Asegurarse de que los nombres de columnas están bien
costos.columns = costos.columns.str.strip()  # quitar espacios accidentales
costos["cobraSoloExitoso"] = costos["cobraSoloExitoso"].astype(bool)

# Unir al dataframe de cobros por 'idEmisora'
cobros = cobros.merge(costos[["idEmisora", "costo", "cobraSoloExitoso"]], on="idEmisora", how="left")

# Calcular el costo real de cada intento según si se cobra solo por éxito o siempre
def calcular_costo_transaccion(row):
    if row["cobraSoloExitoso"]:
        return row["costo"] if row["esExitoso"] else 0
    else:
        return row["costo"]

cobros["costoTransaccion"] = cobros.apply(calcular_costo_transaccion, axis=1)


In [ ]:
# mostrar entradas con idEmidora = 62
print(cobros[cobros["idEmisora"] == 1].shape)

cobros[cobros["idEmisora"] == 1].head()


In [ ]:
idEmisora = cobros["idEmisora"].unique()
# Contar cuantos diferents idBanco tiene cada idEmisora
idBanco_counts = cobros.groupby("idEmisora")["idBanco"].nunique().reset_index()

idBanco_counts.head(50)

In [ ]:
# Ordenar por crédito y consecutivoCobro
cobros = cobros.sort_values(by=["idCredito", "consecutivoCobro"])

# Calcular número de intento por crédito
cobros["num_intento"] = cobros.groupby("idCredito").cumcount() + 1
cobros["es_segundo_intento"] = cobros["num_intento"] == 2
cobros["num_exitos_previos"] = cobros.groupby("idCredito")["esExitoso"].cumsum() - cobros["esExitoso"].astype(int)
cobros["num_intentos_previos"] = cobros["num_intento"] - 1

cobros.head()

In [ ]:
# Proporcion de intentos exitosos segun columna esExitoso 
exitosos = cobros[cobros["esExitoso"] == True]
fallidos = cobros[cobros["esExitoso"] == False]
print("Proporción de intentos exitosos:", len(exitosos) / (len(exitosos) + len(fallidos)))



In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Asegúrate de que `cobros` esté cargado y contenga estas columnas:
# idEmisora, idBanco, montoExigible, montoCobrado, esExitoso

# === 1. Tasa de Éxito por Estrategia (idEmisora) ===
exito_por_estrategia = cobros.groupby("idEmisora")["esExitoso"].agg(["count", "sum"])
exito_por_estrategia["tasa_exito"] = exito_por_estrategia["sum"] / exito_por_estrategia["count"]
exito_por_estrategia = exito_por_estrategia.sort_values("tasa_exito", ascending=False)

plt.figure(figsize=(12, 5))
plt.bar(exito_por_estrategia.index.astype(str), exito_por_estrategia["tasa_exito"])
plt.title("Tasa de Éxito por Estrategia (idEmisora)")
plt.xlabel("idEmisora")
plt.ylabel("Tasa de Éxito")
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

# === 2. Tasa de Éxito por Banco y Estrategia ===
exito_banco_estrategia = cobros.groupby(["idBanco", "idEmisora"])["esExitoso"].agg(["count", "sum"])
exito_banco_estrategia["tasa_exito"] = exito_banco_estrategia["sum"] / exito_banco_estrategia["count"]
pivot_be = exito_banco_estrategia.reset_index().pivot(index="idEmisora", columns="idBanco", values="tasa_exito")

pivot_be.plot(kind="bar", figsize=(12, 6))
plt.title("Tasa de Éxito por Estrategia y Banco")
plt.xlabel("idEmisora")
plt.ylabel("Tasa de Éxito")
plt.xticks(rotation=90)
plt.legend(title="idBanco")
plt.tight_layout()
plt.show()

# === 3. Tasa de Recuperación de Monto por Estrategia ===
monto_estrategia = cobros.groupby("idEmisora")[["montoExigible", "montoCobrado"]].sum()
monto_estrategia["tasa_recuperacion"] = monto_estrategia["montoCobrado"] / monto_estrategia["montoExigible"]
monto_estrategia = monto_estrategia.sort_values("tasa_recuperacion", ascending=False)

plt.figure(figsize=(12, 5))
plt.bar(monto_estrategia.index.astype(str), monto_estrategia["tasa_recuperacion"])
plt.title("Tasa de Recuperación de Monto por Estrategia")
plt.xlabel("idEmisora")
plt.ylabel("Tasa de Recuperación")
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

# %%
# === 4. Análisis de SEGUNDO intento de cobranza ===
segundos = cobros[cobros["es_segundo_intento"] == True]

# Tasa de éxito en el segundo intento
tasa_exito_segundo = segundos["esExitoso"].mean()
print(f"Tasa de éxito en el segundo intento: {tasa_exito_segundo:.2%}")

# Distribución de respuestas en segundo intento
print("\nDistribución de respuestas del banco en segundo intento:")
print(segundos["respuestaCategoria"].value_counts(normalize=True))

# Comparar estrategias más comunes en segundo intento
estrategias_segundo = segundos.groupby("idEmisora")["esExitoso"].agg(["count", "sum"])
estrategias_segundo["tasa_exito"] = estrategias_segundo["sum"] / estrategias_segundo["count"]
estrategias_segundo = estrategias_segundo.sort_values("tasa_exito", ascending=False)

plt.figure(figsize=(12, 5))
plt.bar(estrategias_segundo.index.astype(str), estrategias_segundo["tasa_exito"])
plt.title("Tasa de Éxito por Estrategia en Segundo Intento")
plt.xlabel("idEmisora")
plt.ylabel("Tasa de Éxito en Segundo Intento")
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

# %%
# === 5. Análisis de intentos acumulados antes de éxito ===
# Solo intentos exitosos
exitos = cobros[cobros["esExitoso"] == True]

# Para cada crédito exitoso, ¿en qué intento lo logró?
primer_exito = exitos.groupby("idCredito")["num_intento"].min()

plt.figure(figsize=(12, 5))
plt.hist(primer_exito, bins=range(1, primer_exito.max()+2), edgecolor='black')
plt.title("Intento en el que se logró el primer cobro exitoso")
plt.xlabel("Número de intento (num_intento)")
plt.ylabel("Número de créditos")
plt.xticks(range(1, primer_exito.max()+1))
plt.tight_layout()
plt.show()


In [ ]:
# %%
# Agrupar métricas por estrategia
estrategias_eval = cobros.groupby("idEmisora").agg(
    total_intentos=("idCredito", "count"),
    total_exitos=("esExitoso", "sum"),
    monto_total_cobrado=("montoCobrado", "sum"),
    costo_total=("costoTransaccion", "sum")
)

# Métricas adicionales
estrategias_eval["tasa_exito"] = estrategias_eval["total_exitos"] / estrategias_eval["total_intentos"]
estrategias_eval["roi"] = estrategias_eval["monto_total_cobrado"] / estrategias_eval["costo_total"]
estrategias_eval["efectividad_ajustada"] = (estrategias_eval["monto_total_cobrado"] - estrategias_eval["costo_total"]) / estrategias_eval["total_intentos"]

estrategias_eval = estrategias_eval.sort_values("efectividad_ajustada", ascending=False)
estrategias_eval.reset_index(inplace=True)

# Mostrar top 10 estrategias
estrategias_eval.head(50)


In [ ]:
# %%
# Gráfico: Efectividad ajustada
plt.figure(figsize=(12, 5))
plt.bar(estrategias_eval["idEmisora"].astype(str), estrategias_eval["efectividad_ajustada"])
plt.title("Efectividad Ajustada por Estrategia (MXN neto por intento)")
plt.xlabel("idEmisora")
plt.ylabel("Efectividad Ajustada")
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

# Gráfico: ROI vs Tasa de Éxito
plt.figure(figsize=(10, 6))
plt.scatter(estrategias_eval["tasa_exito"], estrategias_eval["roi"])
for i, row in estrategias_eval.iterrows():
    plt.text(row["tasa_exito"], row["roi"], str(row["idEmisora"]), fontsize=8)
plt.xlabel("Tasa de Éxito")
plt.ylabel("ROI (Cobrado / Costo)")
plt.title("ROI vs Tasa de Éxito por Estrategia")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# %%
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, precision_recall_curve
import seaborn as sns
import matplotlib.pyplot as plt

# Variables para el modelo, incluyendo información financiera
vars_modelo = [
    "idEmisora", "montoExigible", "montoCobrar",
    "num_intentos_previos", "num_exitos_previos", "es_segundo_intento",
    "costoTransaccion", "cobraSoloExitoso"
]

# Preparar X e y
X = cobros[vars_modelo]
y = cobros["esExitoso"].astype(int)

# One-hot encoding
X_encoded = pd.get_dummies(X, columns=["idEmisora", "cobraSoloExitoso"])

# Dividir en train/test
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.3, random_state=42, stratify=y
)

# Modelo con ajuste por desbalance
clf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight="balanced")
clf.fit(X_train, y_train)

# Predicciones
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)[:, 1]

# === Evaluación ===
print("\n=== Classification Report (con class_weight='balanced') ===")
print(classification_report(y_test, y_pred))

print(f"AUC: {roc_auc_score(y_test, y_proba):.4f}")

# === Matriz de confusión ===
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Fallido", "Exitoso"], yticklabels=["Fallido", "Exitoso"])
plt.title("Matriz de Confusión")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.tight_layout()
plt.show()

# === Curva de precisión vs recall ===
precision, recall, _ = precision_recall_curve(y_test, y_proba)
plt.figure(figsize=(10, 5))
plt.plot(recall, precision)
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Curva Precision-Recall")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
def simular_estrategias(cobros, modelo, X_train, n_creditos=1000, umbral=0.05):
    import pandas as pd
    import numpy as np

    # Selección de créditos únicos
    creditos_muestra = cobros["idCredito"].drop_duplicates().sample(n_creditos, random_state=42)
    ultimos_intentos = cobros[cobros["idCredito"].isin(creditos_muestra)]
    ultimos_intentos = ultimos_intentos.sort_values("consecutivoCobro").groupby("idCredito").tail(1)

    # Estrategias válidas únicas
    estrategias_validas = cobros[["idEmisora", "costo", "cobraSoloExitoso"]].drop_duplicates()

    # Cross join explícito
    simulacion = ultimos_intentos.assign(key=1).merge(
        estrategias_validas.assign(key=1), on="key", how="outer", suffixes=("_credito", "_estrategia")
    ).drop("key", axis=1)

    # Costo estimado
    simulacion["costoTransaccion_estimado"] = simulacion["costo_estrategia"]

    # Variables del modelo
    variables_modelo = [
        "idEmisora_estrategia", "montoExigible", "montoCobrar",
        "num_intentos_previos", "num_exitos_previos", "es_segundo_intento",
        "costoTransaccion_estimado", "cobraSoloExitoso_estrategia"
    ]

    # Preparar X para predicción
    X_simulacion = simulacion[variables_modelo].copy()
    X_simulacion_encoded = pd.get_dummies(
        X_simulacion,
        columns=["idEmisora_estrategia", "cobraSoloExitoso_estrategia"]
    )

    # Alinear con entrenamiento
    for col in set(X_train.columns) - set(X_simulacion_encoded.columns):
        X_simulacion_encoded[col] = 0
    X_simulacion_encoded = X_simulacion_encoded[X_train.columns]

    # Predicción
    simulacion["proba_exito"] = modelo.predict_proba(X_simulacion_encoded)[:, 1]

    # Escenario A: Minimizar Comisión
    sim_a = simulacion[
        (simulacion["proba_exito"] >= umbral) &
        (simulacion["respuestaCategoria"] != "cuenta_invalida")
    ]
    mejores_a = sim_a.sort_values(["idCredito", "costoTransaccion_estimado"]).groupby("idCredito").first().reset_index()
    mejores_a["escenario"] = "Minimizar Comisión"

    # Escenario B: Maximizar Recuperación
    p75_monto = simulacion["montoExigible"].quantile(0.75)
    sim_b = simulacion[
        (simulacion["montoExigible"] >= p75_monto) &
        (simulacion["num_intento"].isin([2, 5]))
    ]
    mejores_b = sim_b.sort_values(["idCredito", "proba_exito"], ascending=[True, False]).groupby("idCredito").first().reset_index()
    mejores_b["escenario"] = "Maximizar Cobranza"

    # Unir ambos
    resultados = pd.concat([mejores_a, mejores_b])
    resultados = resultados[[
        "idCredito", "idEmisora_estrategia", "proba_exito",
        "costoTransaccion_estimado", "escenario"
    ]].reset_index(drop=True)

    return resultados


In [ ]:
numeEntriesCobros = cobros.drop_duplicates(subset=["idCredito"]).shape[0]
print(f"Total de entradas en cobros: {numeEntriesCobros}")
resultados = simular_estrategias(cobros, clf, X_train, n_creditos=numeEntriesCobros, umbral=0.01)
# Mostrar resultados de la simulación, la cantidad de cada idEmisora_estrategia
print("\nResultados de la simulación:")
print(resultados["idEmisora_estrategia"].value_counts())


In [ ]:
def simular_estrategias_por_etapas_en_batch(cobros, modelo, X_train, umbral=0.05, batch_size=100_000, verbose=True):
    import pandas as pd
    import numpy as np

    estrategias_validas = cobros[["idEmisora", "costo", "cobraSoloExitoso"]].drop_duplicates()
    total_rows = cobros.shape[0]
    resultados = []

    for start in range(0, total_rows, batch_size):
        end = min(start + batch_size, total_rows)
        if verbose:
            print(f"🔄 Procesando filas {start} a {end} de {total_rows}...")

        intentos_batch = cobros.iloc[start:end].copy()

        # Cross join con estrategias
        simulacion = intentos_batch.assign(key=1).merge(
            estrategias_validas.assign(key=1), on="key", how="outer", suffixes=("_credito", "_estrategia")
        ).drop("key", axis=1)

        # Estimar costo
        simulacion["costoTransaccion_estimado"] = simulacion["costo_estrategia"]

        # Variables
        variables_modelo = [
            "idEmisora_estrategia", "montoExigible", "montoCobrar",
            "num_intentos_previos", "num_exitos_previos", "es_segundo_intento",
            "costoTransaccion_estimado", "cobraSoloExitoso_estrategia"
        ]

        X_simulacion = simulacion[variables_modelo].copy()
        X_simulacion_encoded = pd.get_dummies(
            X_simulacion,
            columns=["idEmisora_estrategia", "cobraSoloExitoso_estrategia"]
        )

        # Alinear con modelo
        for col in set(X_train.columns) - set(X_simulacion_encoded.columns):
            X_simulacion_encoded[col] = 0
        X_simulacion_encoded = X_simulacion_encoded[X_train.columns]

        # Predicción
        simulacion["proba_exito"] = modelo.predict_proba(X_simulacion_encoded)[:, 1]

        # Escenario A
        sim_a = simulacion[
            (simulacion["proba_exito"] >= umbral) &
            (simulacion["respuestaCategoria"] != "cuenta_invalida")
        ]
        mejores_a = sim_a.sort_values(["idCredito", "consecutivoCobro", "costoTransaccion_estimado"]).groupby(
            ["idCredito", "consecutivoCobro"]
        ).first().reset_index()
        mejores_a["escenario"] = "Minimizar Comisión"

        # Escenario B
        p75_monto = simulacion["montoExigible"].quantile(0.75)
        sim_b = simulacion[
            (simulacion["montoExigible"] >= p75_monto) &
            (simulacion["num_intento"].isin([2, 5]))
        ]
        mejores_b = sim_b.sort_values(["idCredito", "consecutivoCobro", "proba_exito"], ascending=[True, True, False]).groupby(
            ["idCredito", "consecutivoCobro"]
        ).first().reset_index()
        mejores_b["escenario"] = "Maximizar Cobranza"

        # Unir batch
        batch_resultados = pd.concat([mejores_a, mejores_b], ignore_index=True)
        resultados.append(batch_resultados)

    # Unir todos los resultados
    resultados_df = pd.concat(resultados, ignore_index=True)
    resultados_df = resultados_df[[
        "idCredito", "consecutivoCobro", "idEmisora_estrategia", "proba_exito",
        "costoTransaccion_estimado", "escenario"
    ]]

    return resultados_df


In [ ]:
resultados_finales = simular_estrategias_por_etapas_en_batch(cobros, clf, X_train, umbral=0.05, batch_size=100_000)
display.display(resultados_finales.head(30))
